# Generating setList for Pathway Analysis
- Using the MSigDB C2 Canonical Pathways
- Downloaded Oct 5 2021 (v7.4)
- For PD Burdens project

## Install the necessary packages

In [1]:
# pip install pyannotables
from pyannotables import tables, homology_convert
import pandas as pd
import csv 
import numpy as np

# Print out the tables available in the package 
tables = tables()
#sorted(tables.keys())

# Make list of genes incoming from the .gmt files

In [2]:
## Make list of all possible genes in incoming file
! awk '{$1=$2=""; print $0}' c2.cp.v7.4.symbols.gmt > list_of_genes.txt # Copy everything other than the first 2 columns 
! tr ' ' '\n' < list_of_genes.txt > pre_file_list_of_genes.txt # Convert the spaces between each gene to new lines 
! sed '/^$/d' pre_file_list_of_genes.txt > file_list_of_genes.txt # Remove any empty rows from the file to prevent problems later 

# Clean up
! rm list_of_genes.txt
! rm pre_file_list_of_genes.txt

## Read in file
with open('file_list_of_genes.txt') as f:
    all_genes = f.read().splitlines()

# Creating the dictionaries of hg38 coordinates 
- [x] Annotables (Ensembl 100)
- [x] refFlat hg38 
- [x] Biomart hg38 (HGNC symbols)
- [x] Manually adding in any missing genes

## Read in the annotables

In [3]:
# Grab the hg38 human build most recent Ensembl info 
hg38_table = tables['homo_sapiens-GRCh38-ensembl100'].copy()

# Abbreviate to keep the columns we need + reset index to prevent merging errors 
hg38_table_annotables_abbrev = hg38_table[['gene_name', 'Chromosome', 'Start', 'End']].copy().reset_index(drop=True)

# Make sure to only keep chr1-22, X, and Y and none of the bullshit prefixes that have underscores in them
hg38_table_annotables_abbrev = hg38_table_annotables_abbrev.loc[~hg38_table_annotables_abbrev['Chromosome'].str.contains(r'_')]  

# Make one column of the coordinates in the format we will need for the setFile
hg38_table_annotables_abbrev['Coordinates'] = hg38_table_annotables_abbrev['Chromosome'].astype(str) + ":" + hg38_table_annotables_abbrev['Start'].astype(str) + "-" + hg38_table_annotables_abbrev['End'].astype(str)

# Keep the first instance of the gene symbol 
hg38_table_annotables_abbrev.drop_duplicates(subset=['gene_name'], keep='first', inplace=True)

hg38_table_annotables_abbrev.head()

,gene_name,Chromosome,Start,End,Coordinates
0,TSPAN6,X,100627107,100639991,X:100627107-100639991
3,TNMD,X,100584935,100599885,X:100584935-100599885
4,DPM1,20,50934866,50958555,20:50934866-50958555
8,SCYL3,1,169849630,169894267,1:169849630-169894267
10,C1orf112,1,169662006,169854080,1:169662006-169854080


In [4]:
# Generate a dictionary with the keys being the gene symbol and the coordinates being the values 
annotables_dict = dict(zip(hg38_table_annotables_abbrev.gene_name, hg38_table_annotables_abbrev.Coordinates))
len(annotables_dict)

60650

### Compare annotables dictionary to the genes incoming

In [5]:
## Compare with dictionary and see what genes are missing
missing_genes = [] # Initialize empty list 
for gene in all_genes:
    if gene not in annotables_dict:
        missing_genes.append(gene)

unique_missing_genes = list(set(missing_genes)) # Make unique list of genes missing 
print(unique_missing_genes)
len(unique_missing_genes)
    # Looks like there are 42 missing genes after using annotables
    # Let's supplement this with other databases

['KIR2DS3', 'UQCR10P1', 'CEP20', 'BPNT2', 'KIR2DL5A', 'POLR1H', 'DNAAF6', 'IRAG2', 'CIBAR1', 'SKP1P2', 'OR8U9', 'OR8U8', 'DYNLT2', 'OR8U3', 'IHO1', 'KIR2DL2', 'SSPOP', 'POLR1G', 'PDCD6-AHRR', 'HLA-DRB4', 'KIR2DS5', 'KATNIP', 'TAS2R43', 'POLR1F', 'LILRA3', 'CEP43', 'GARRE1', 'ZZZ3', 'CCL3L3', 'MIR124-1HG', 'DYNLT2B', 'DYNC2I2', 'C4B_2', 'DYNC2I1', 'KIR2DS2', 'GFUS', 'VDAC2P5', 'IRAG1', 'KIR2DS1', 'GSTT1', 'KIR3DS1', 'HLA-DRB3']


42

## Read in the Biomart file
- Downloaded Oct 7th 
- hg38 coordinates 
- HGNC symbols

In [6]:
## Read in the biomart file 
biomart = pd.read_csv("mart_export.txt", sep="\t")
biomart.columns = ['Start','End','Chromosome','gene_name','entrezID'] # Force column names 

# Make sure to only keep chr1-22, X, and Y and none of the bullshit prefixes that have underscores in them
biomart = biomart.loc[~biomart['Chromosome'].str.contains(r'_')]  

# Make coordinates 
biomart['Coordinates'] = biomart['Chromosome'].astype(str) + ":" + biomart['Start'].astype(str) + "-" + biomart['End'].astype(str)

# Keep the first instance of the gene symbol 
biomart.drop_duplicates(subset=['gene_name'], keep='first', inplace=True)

# Make a dictionary of the biomart values 
biomart_dict = dict(zip(biomart.gene_name, biomart.Coordinates))

In [7]:
# Merge 2 dictionaries, but prioritize the annotables one
# declaring priority order
prioritize_dictionaries = {1 : annotables_dict, 2: biomart_dict}

# Time to merge
annotables_biomart_dict = prioritize_dictionaries[2].copy()
for key, val in prioritize_dictionaries[1].items():
    annotables_biomart_dict[key] = val

In [8]:
## Sanity check - make sure no weird bullshit
# Pull out the coordinates for GBA 
search_key = 'GBA'
dict(filter(lambda item: search_key in str(item[0]), annotables_biomart_dict.items()))

    # Looks good!

{'GBA3': '4:22692913-22819575',
 'GBA2': '9:35736865-35749228',
 'GBAP1': '1:155213820-155227422',
 'GBA': '1:155234451-155244699'}

### Compare Biomart dictionary to the genes incoming

In [9]:
## Compare with dictionary and see what genes are missing
missing_genes = [] # Initialize empty list 
for gene in all_genes:
    if gene not in annotables_biomart_dict:
        missing_genes.append(gene)

unique_missing_genes = list(set(missing_genes)) # Make unique list of genes missing 
print(unique_missing_genes)
len(unique_missing_genes)
    # Still 14 missing genes, let's add refFlat next

['KIR2DS5', 'KIR2DS3', 'OR8U8', 'KIR2DL2', 'OR8U9', 'C4B_2', 'KIR2DL5A', 'KIR2DS2', 'LILRA3', 'KIR2DS1', 'HLA-DRB4', 'GSTT1', 'KIR3DS1', 'HLA-DRB3']


14

## Read in the refFlat file
- From UCSC 
- hg38 coordinates

In [10]:
## Read in refFlat file 
refFlat = pd.read_csv("refFlat_hg38_unique_geneChrStartStop.txt", sep="\t", header=None)
refFlat.columns = ['gene_name', 'Chromosome','Start','End'] # Force column names 
refFlat['Chromosome'] = refFlat['Chromosome'].str.replace("chr", "") # Remove chr prefix 

# Make sure to only keep chr1-22, X, and Y and none of the bullshit prefixes that have underscores in them
refFlat = refFlat.loc[~refFlat['Chromosome'].str.contains(r'_')]  

# Make coordinates 
refFlat['Coordinates'] = refFlat['Chromosome'].astype(str) + ":" + refFlat['Start'].astype(str) + "-" + refFlat['End'].astype(str)

# Keep the first instance of the gene symbol 
refFlat.drop_duplicates(subset=['gene_name'], keep='first', inplace=True)

# Make a dictionary of the biomart values 
refFlat_dict = dict(zip(refFlat.gene_name, refFlat.Coordinates))

refFlat.head()

,gene_name,Chromosome,Start,End,Coordinates
0,A1BG,19,58345182,58353492,19:58345182-58353492
1,A1BG-AS1,19,58351969,58355183,19:58351969-58355183
2,A1CF,10,50799408,50885675,10:50799408-50885675
3,A2M,12,9067707,9115919,12:9067707-9115919
4,A2M-AS1,12,9065176,9068055,12:9065176-9068055


In [11]:
# Merge 2 dictionaries, but prioritize the annotables one
# declaring priority order
prioritize_dictionaries = {1 : annotables_biomart_dict, 2: refFlat_dict}

# Time to merge
annotables_biomart_refFlat_dict = prioritize_dictionaries[2].copy()
for key, val in prioritize_dictionaries[1].items():
    annotables_biomart_refFlat_dict[key] = val

### Compare refFlat dictionary to the genes incoming

In [12]:
## Compare with dictionary and see what genes are missing
missing_genes = [] # Initialize empty list 
for gene in all_genes:
    if gene not in annotables_biomart_refFlat_dict:
        missing_genes.append(gene)

unique_missing_genes = list(set(missing_genes)) # Make unique list of genes missing 
print(unique_missing_genes)
len(unique_missing_genes)
    # Still 12 missing genes...

['KIR2DS5', 'KIR2DS3', 'KIR2DL2', 'OR8U9', 'KIR2DL5A', 'KIR2DS2', 'LILRA3', 'KIR2DS1', 'HLA-DRB4', 'GSTT1', 'KIR3DS1', 'HLA-DRB3']


12

## Adding in missing genes 

In [13]:
# No other way around this, adding the 12 via NCBI 
# Most of these are immuno- or olfactory genes, so they mutate and seem to map multiple places and have multiple complements - probably why we had issues with them

    # LILRA3 (19:270964-275379)
    # KIR2DS3 (19:150177-164532)
    # KIR3DS1 (19:58332-72919)
    # HLA-DRB4 (6:3840427-3855395)
    # KIR2DL5A (19:46835-5629)
    # KIR2DS1 (19:13877-27891)
    # KIR2DL2 (19:106541-121079)
    # KIR2DS5 (19:29858-44878)
    # KIR2DS2 (19:122955-137265)
    # OR8U9 (11:193979-194908)
    # GSTT1 (22:270308-278486) 
    # HLA-DRB3 (6:3934021-3947089)
    
# Reading it in as a tab-delimited .txt file 
extra_genes = pd.read_csv("adding_missing.txt", header=None, sep="\t")
extra_genes.columns = ['gene_name', 'Coordinates']

# Make a dictionary of the missing values 
extra_genes_dict = dict(zip(extra_genes.gene_name, extra_genes.Coordinates))

# Merge 2 dictionaries, but prioritize the annotables one
# declaring priority order
prioritize_dictionaries = {1 : annotables_biomart_refFlat_dict, 2: extra_genes_dict}

# Time to merge
annotables_biomart_refFlat_extra_dict = prioritize_dictionaries[2].copy()
for key, val in prioritize_dictionaries[1].items():
    annotables_biomart_refFlat_extra_dict[key] = val

In [14]:
## Sanity check - make sure no weird bullshit
# Pull out the coordinates for GBA 
search_key = 'LRRK2'
dict(filter(lambda item: search_key in str(item[0]), annotables_biomart_refFlat_dict.items()))

{'LRRK2': '12:40196743-40369285', 'LRRK2-DT': '12:40186008-40224915'}

### Compare full dictionary to the genes incoming

In [15]:
## Compare with dictionary and see what genes are missing
missing_genes = [] # Initialize empty list 
for gene in all_genes:
    if gene not in annotables_biomart_refFlat_extra_dict:
        missing_genes.append(gene)

unique_missing_genes = list(set(missing_genes)) # Make unique list of genes missing 
print(unique_missing_genes)
len(unique_missing_genes)
    # No more missing genes 

[]


0

# Replace symbols with coordinates

In [16]:
## Make a function that maps the dictionary values 
def replace(list, dictionary):
    return [dictionary.get(item, item) for item in list]

In [17]:
# Make an array of the weird chromsome prefixes to remove (if there are any)
prefixes = ('CHR_')

# Initialize an empty list 
list_of_pathways = []
list_of_error_pathways = [] 

# Read in the MSigDB .gmt file 
# Loop through the c2 file 
with open('c2.cp.v7.4.symbols.gmt', 'r') as f:
    reader = csv.reader(f, dialect='excel', delimiter='\t')
    for row in reader:
        del row[1] # Remove the 2nd element, the ones with the website link 
        number_of_genes = len(row)
        row2 = replace(row, annotables_biomart_refFlat_extra_dict) # Map the coordinates to the gene symbols
        row3 = [element for element in row2 if not element.startswith(prefixes)] # Remove any of those weird prefixes 
        number_of_found_genes = len(row2)
        if number_of_genes != number_of_found_genes:
            list_of_error_pathways.append(row3)
        #print(row3)
        list_of_pathways.append(row3) # Add to the list of lists 

In [18]:
## Sanity check, see if any of the now converted pathways have more or less genes than the input file 
len(list_of_error_pathways)

0

# Save out the files

## Save out new setFile

In [19]:
# Save out the file 
with open('c2.cp.v7.4.symbols.setFile.Oct52021.txt', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(list_of_pathways)

In [20]:
# No idea how to do this in Python but use bash to replace the FIRST comma with a tab and remove the temp file
! sed 's/,/'$'\t''/' c2.cp.v7.4.symbols.setFile.Oct52021.txt > c2.cp.v7.4.symbols.setFile.Oct52021.final.txt
! rm c2.cp.v7.4.symbols.setFile.Oct52021.txt
! wc -l c2.cp.v7.4.symbols.setFile.Oct52021.final.txt

    2922 c2.cp.v7.4.symbols.setFile.Oct52021.final.txt


## Save out coordinates file

In [21]:
# Save out these coordinates to limit the amount of work next time to make a dictionary...
df = pd.DataFrame(list(annotables_biomart_refFlat_extra_dict.items()), columns=['gene_name', 'Coordinates'])
df.to_csv("hg38_refFlat_annotables_BioMart_genes_coordinates_OCT2021.txt", header=True, index=False, sep="\t")

# Sanity Checks
- [x] Check to see if GBA is in pathways and showing up in setFile 
- [x] Check to see no weird prefixes (_alt, CHR_ etc.)
- [x] Check to see if symbols are matching up to expected coordinates 
- [x] Double check that some genes might not be unique because there are aliases and will have matching coordinates

In [22]:
! grep -w "GBA"  c2.cp.v7.4.symbols.gmt | head -3

KEGG_OTHER_GLYCAN_DEGRADATION	http://www.gsea-msigdb.org/gsea/msigdb/cards/KEGG_OTHER_GLYCAN_DEGRADATION	ENGASE	GLB1	MANBA	MAN2B1	GBA	NEU4	NEU2	NEU1	FUCA1	FUCA2	AGA	MAN2C1	MAN2B2	NEU3	HEXB	HEXA
KEGG_SPHINGOLIPID_METABOLISM	http://www.gsea-msigdb.org/gsea/msigdb/cards/KEGG_SPHINGOLIPID_METABOLISM	GAL3ST1	SGPP2	GLB1	GALC	SGMS2	GBA	SPHK2	NEU2	PLPP3	NEU1	ACER2	UGCG	DEGS2	ARSA	SPHK1	SGPL1	NEU3	SMPD4	SGMS1	ACER3	CERK	SMPD2	ENPP7	DEGS1	SGPP1	NEU4	GLA	ACER1	PLPP1	ASAH1	PLPP2	SMPD1	B4GALT6	SPTLC2	SPTLC1	SMPD3	KDSR	UGT8	ASAH2
KEGG_LYSOSOME	http://www.gsea-msigdb.org/gsea/msigdb/cards/KEGG_LYSOSOME	PLA2G15	AP3B2	GGA1	SLC11A1	PPT1	MFSD8	CTSZ	NAGPA	IGF2R	ARSG	ATP6V0A1	CTSW	ATP6V0B	LGMN	CTSS	AP4S1	LAMP2	ATP6AP1	LAPTM5	AP1S3	CLTCL1	AP4B1	AP3B1	LAMP3	ATP6V0A4	CTSL	ABCB9	AP1B1	CTSK	SLC11A2	GNPTAB	SMPD1	CTSH	AP1G1	CTSG	CTSE	LAPTM4A	GM2A	LAMP1	CTSO	GGA2	CTSV	AP3M1	CTSC	GBA	AP3D1	CD164	ATP6V1H	HGSNAT	ABCA2	DNASE2B	AGA	AP3M2	ATP6V0D2	ARSA	CTSD	CTSB	ARSB	TCIRG1	GNS	PPT2	DNASE2	SORT1	ASAH1	AP4E1	GALNS	AP4M1	

In [23]:
! grep -w 'KEGG_OTHER_GLYCAN_DEGRADATION' c2.cp.v7.4.symbols.setFile.Oct52021.final.txt

KEGG_OTHER_GLYCAN_DEGRADATION	17:79074823-79088599,3:32996608-33097202,4:102630769-102760994,19:12646510-12666742,1:155234451-155244699,2:241808824-241817413,2:233032671-233035057,6:31857658-31862822,1:23845076-23868294,6:143494811-143511720,4:177430773-177442437,15:75355206-75368612,4:6575188-6623362,11:74988278-75018893,5:74640022-74722647,15:72340923-72376420


In [24]:
# Check that GBA is where it is supposed to be 
! grep "1:155234451-155244699" c2.cp.v7.4.symbols.setFile.Oct52021.final.txt | head -3

KEGG_OTHER_GLYCAN_DEGRADATION	17:79074823-79088599,3:32996608-33097202,4:102630769-102760994,19:12646510-12666742,1:155234451-155244699,2:241808824-241817413,2:233032671-233035057,6:31857658-31862822,1:23845076-23868294,6:143494811-143511720,4:177430773-177442437,15:75355206-75368612,4:6575188-6623362,11:74988278-75018893,5:74640022-74722647,15:72340923-72376420
KEGG_SPHINGOLIPID_METABOLISM	22:30554634-30574665,2:222424542-222562621,3:32996608-33097202,14:87837819-87993665,4:107824562-107915047,1:155234451-155244699,19:48619290-48630717,2:233032671-233035057,1:56494760-56645301,6:31857658-31862822,9:19409008-19452505,9:111896813-111935369,14:100143956-100159645,22:50622753-50628173,17:76376583-76387860,10:70815947-70881184,11:74988278-75018893,2:130151391-130182750,10:50305585-50625163,11:76860866-77026797,22:46684409-46738252,6:109440723-109443919,17:79730942-79742219,1:224175755-224193441,14:63684215-63728065,2:241808824-241817413,X:101393272-101408012,19:6306141-6333612,5:55424853-5

In [25]:
! grep "_alt" c2.cp.v7.4.symbols.setFile.Oct52021.final.txt | head -3 # Nothing

In [27]:
! grep "CHR_" c2.cp.v7.4.symbols.setFile.Oct52021.final.txt | head -3 # Nothing

In [31]:
## KEGG_OTHER_GLYCAN_DEGRADATION
# ENGASE
# GLB1
# MANBA
# MAN2B1
# GBA
# NEU4
# NEU2
# NEU1
# FUCA1
# FUCA2
# AGA
# MAN2C1
# MAN2B2
# NEU3
# HEXB
# HEXA

## KEGG_OTHER_GLYCAN_DEGRADATION
# 17:79074823-79088599
# 3:32996608-33097202
# 4:102630769-102760994
# 19:12646510-12666742
# 1:155234451-155244699
# 2:241808824-241817413
# 2:233032671-233035057
# 1:23845076-23868294
# 6:143494811-143511720
# 4:177430773-177442437
# 15:75355206-75368612
# 4:6575188-6623362
# 11:74988278-75018893
# 5:74640022-74722647
# 15:72340923-72376420

search_key = 'MANBA'
dict(filter(lambda item: search_key in str(item[0]), annotables_biomart_refFlat_dict.items()))

# Spot checks look good, values as expected 

{'MANBA': '4:102630769-102760994', 'MANBAL': '20:37289637-37317260'}

In [28]:
## Check for duplicate coordinates (symbols have multiple aliases sometimes...)
df.loc[df.Coordinates.duplicated(), :]

,gene_name,Coordinates
11958,LOC100130520,17:74560700-74567343
11986,LOC100132287,1:490755-495445
12027,LOC100287834,7:63349069-63351774
12731,LOC102724843,21:5553636-5592004
12736,LOC102724951,21:5553636-5592004
...,...,...
61404,AL158069.2,X:69672478-69672597
61525,AC104843.3,7:152592972-152593091
61561,AC011405.2,5:139012328-139012441
62872,AL589787.1,10:126425929-126460957


In [29]:
df[df['Coordinates'].str.match('1:490755-495445')]

,gene_name,Coordinates
11980,LOC100132062,1:490755-495445
11986,LOC100132287,1:490755-495445
